In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm

In [ ]:
def profit(option_prices, yi, market_price, strategy):
    res = yi * market_price * 1000
    before_options = res
#     print('Income from selling is:', res)
    for i in range(14):
        strike, typ = options[i]
#         print(strike, typ)
        res += strategy[i] * calculate_profit(strike, market_price, typ)
        res -= option_prices[i] * strategy[i]
#     if res-before_options > 0:
#         print(res-before_options, strategy)
#     print("profit on options:", res-before_options)
    return res

In [ ]:
number_of_random_strategies = 10000
best_quantile_10 = 0
min_quantile_10 = 1320000
min_quantile_25 = 1400000
for i in tqdm(range(number_of_random_strategies)):
#     strat = np.random.randint(100, size=14)
    strat = np.hstack((np.random.randint(2000, size=7), np.zeros(7)))
    if (best_strat != []):
      strat = np.round(closer_sbs(best_strat, strat, 2))
#     strat = [1000, 1000, 1000, 100, 1000, 100, 0, 0, 0, 0, 0, 0, 0, 0]
    curr_profits = []
    for j in range(1000):
        curr_strat_profit = profit(prices_list, sc1_list['yield [thousands of bushels]'][j], sc1_list['price [$/bushel]'][j], strat)
#         print(curr_strat_profit)
#         print(sc1_list['yield [thousands of bushels]'][j])
#         print(sc1_list['price [$/bushel]'][j])
        curr_profits.append(curr_strat_profit)
    curr_expectet_profit = np.mean(curr_profits)
    quantile_10 = np.quantile(curr_profits, 0.1)
    quantile_25 = np.quantile(curr_profits, 0.25)
    if curr_expectet_profit > best_profit and quantile_10 > min_quantile_10 and quantile_25 > min_quantile_25:
        best_profit = curr_expectet_profit
        best_strat = strat
        print(quantile_10)
        print(quantile_25)
        print('New best strategy at iteration ',i,' : ', strat, 'Expected Profit:', curr_expectet_profit)